In [11]:
import pandas as pd
import numpy as np
import re
import nltk
import string
import sklearn
from collections import Counter
from nltk.stem import WordNetLemmatizer, PorterStemmer, SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso, RidgeCV, RidgeClassifierCV
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
#логистическая и линейная регрессия
alli = pd.read_csv('ffwitems.csv')
sale = pd.read_csv('ffwsale.csv')

def get_wordnet_pos(tag): # вернуть ворднету часть речи(у нлтк поз тега и ворднета разные теги) / я спиздил откуда-то
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# я сразу сделал lower тк есть описания фулл-капс, но по-хорошему наверное кастить lower только на фулл капс
def preprocess(text):
    txt = text.lower().split()
    for word in txt:
        if word.isdigit() or word in stopwords.words('english'): 
            txt.remove(word)
    text = ' '.join(txt)

    lemmatizer = WordNetLemmatizer()

    pos = nltk.pos_tag(word_tokenize(text))
    lemmatized = [lemmatizer.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(pos)]
    return " ".join(lemmatized)

In [3]:
df_train = pd.DataFrame()

In [4]:
df_train['description'] = alli['Short_descr'].apply(lambda x: preprocess(x))

In [5]:
df_train['res'] = alli['cat']
df_train['res'].replace({'trainers' : 'shoes'}, inplace=True) # в шуз попадаются кроссовки - сильно снижает качество
X_train, X_test, y_train, y_test = train_test_split(df_train["description"],df_train["res"],test_size=0.6)
tfidf_vectorizer = TfidfVectorizer()
X_train_vectors = tfidf_vectorizer.fit_transform(X_train) 
X_test_vectors = tfidf_vectorizer.transform(X_test)

In [15]:
lr_tfidf = LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2')
grid = {'C':np.logspace(-10, 10, 20), "penalty":["l1","l2"]}
lr_cv=GridSearchCV(lr_tfidf,grid,cv=10)
lr_cv.fit(X_train_vectors, y_train)  
y_predicted = lr_cv.predict(X_test_vectors)

print(classification_report(y_test,y_predicted))
print('Confusion Matrix:',confusion_matrix(y_test, y_predicted))

              precision    recall  f1-score   support

 accessories       0.98      0.98      0.98     10465
        bags       0.99      0.98      0.99     11022
    clothing       1.00      1.00      1.00     47636
   jewellery       0.99      0.99      0.99      6337
       shoes       1.00      1.00      1.00     15379

    accuracy                           0.99     90839
   macro avg       0.99      0.99      0.99     90839
weighted avg       0.99      0.99      0.99     90839

Confusion Matrix: [[10294    77    62    23     9]
 [  160 10839    14     5     4]
 [   50     3 47575     6     2]
 [   28     3    38  6266     2]
 [    5     2    27     2 15343]]


In [7]:
linreg = RidgeClassifierCV(alphas=np.arange(1, 10, 1))
linreg.fit(X_train_vectors, y_train)
predicted = linreg.predict(X_test_vectors)

print(classification_report(y_test,predicted))
print('Confusion Matrix: \n',confusion_matrix(y_test, predicted))

              precision    recall  f1-score   support

 accessories       0.97      0.97      0.97     10470
        bags       0.99      0.98      0.98     10976
    clothing       0.99      1.00      0.99     47770
   jewellery       0.99      0.98      0.98      6393
       shoes       1.00      1.00      1.00     15230

    accuracy                           0.99     90839
   macro avg       0.99      0.98      0.99     90839
weighted avg       0.99      0.99      0.99     90839

Confusion Matrix: 
 [[10171    75   187    27    10]
 [  159 10711    92    11     3]
 [  135     7 47579    42     7]
 [   24     3   124  6235     7]
 [    2     1    62     3 15162]]


In [8]:
print(linreg.alpha_)

1


In [9]:
#метод без машинки, по популярности токенов
df = pd.read_csv('ffwitems.csv')
stop_words = stopwords.words('english')


def prep(w, sw):
    w = w.lower()
    tokenizedw = word_tokenize(w)
    final_tokens = []
    for word in tokenizedw:
        if word in sw or word.isdigit():
            pass
        else:
            final_tokens.append(word)

    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(word) for word in final_tokens]

    return lemmatized

In [12]:
categ = {}
for cat in df['cat'].unique():
    items = df[df['cat'] == cat]
    words = ' '.join(items['Short_descr'])
    categ[cat] = Counter(prep(words, stop_words))

In [13]:
def select_max(words):
    pop = []
    for i in categ.keys():
        temp = 0
        for word in words:
            if word in categ[i].keys():
                temp += int(categ[i][word])
        pop.append((i, temp))
    return max(pop, key = lambda x: x[1])[0]

saledf = pd.read_csv('ffwsale.csv', error_bad_lines=False)
saledf.drop_duplicates(subset='img', inplace=True, ignore_index=True)
saledf['cat'] = saledf['Short_descr'].apply(lambda x: select_max(prep(x, stop_words)))
saledf

C:\Users\cslul\AppData\Local\Temp/ipykernel_3016/2614322722.py:11: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  saledf = pd.read_csv('ffwsale.csv', error_bad_lines=False)


,ffcollection,Brand,Short_descr,Total_Price,Discount,Final_Price,avSizes,img,cat
0,NaN,Palm Angels,logo-print track polo shirt,$420,-50%,$210,"XXS, XS, S, M",https://cdn-images.farfetch-contents.com/16/86...,clothing
1,NaN,Off-White,ankle-tie leather sandals,"$1,605",-35%,"$1,043",See all sizes,https://cdn-images.farfetch-contents.com/17/56...,shoes
2,NaN,Palm Angels,Crash logo-print belt bag,$570,-20%,$456,One Size,https://cdn-images.farfetch-contents.com/16/86...,bags
3,NaN,Off-White,belted mid-length coat,"$2,130",-40%,"$1,278","XXS, M",https://cdn-images.farfetch-contents.com/17/35...,clothing
4,NaN,Alexander McQueen,Tread Slick boots,$950,-50%,$475,See all sizes,https://cdn-images.farfetch-contents.com/16/73...,shoes
...,...,...,...,...,...,...,...,...,...
74716,NaN,Susan Caplan Vintage,1960s Trifari leaf bracelet,$273,-0%,$273,One Size,https://cdn-images.farfetch-contents.com/13/95...,jewellery
74717,NaN,Susan Caplan Vintage,1960s 18kt Gold Plated Brass Avon Bird Earrings,$332,-0%,$332,One Size,https://cdn-images.farfetch-contents.com/13/60...,jewellery
74718,Positively Conscious,Susan Caplan Vintage,1960's Trifari Vintage leaves brooch,$332,-0%,$332,One Size,https://cdn-images.farfetch-contents.com/13/34...,clothing
74719,NaN,Christian Dior,1990s pre-owned oval logo bracelet,$933,-0%,$933,One Size,https://cdn-images.farfetch-contents.com/12/97...,clothing
